In [3]:
#Inbound Process
import paramiko
import sys
import shutil
import time
import posixpath
import datetime
import os

#Today's date
get_date = datetime.datetime.now().strftime("%Y%m%d_%H%M")

#path variables
local = os.path.join(os.getcwd(), 'Processing')
remote = '/Files/ScannedDocs/'
archive = os.path.join(local, "Archive")
 

#Paramiko instancw
client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname = '192.168.64.3', username = 'testuser', password = 'TempPassword1')

with open(f"/Users/homebase/Desktop/Python/Udemy Course/Milestone Projects/OPUS/Processing/Log/Inbound/{get_date}", 'a') as f:
    f.write(f"============= {get_date} Processing =============\n")
    with client.open_sftp() as sftp:
        for folder, sub_folders, files in os.walk(local):
            if os.path.basename(folder) in ['Archive', 'Log', 'Outbound']: #Ignore these folders
                continue
    
            for file in files:
                if file.endswith(".pdf") or file.endswith(".csv"):
                    local_source = os.path.join(folder, file)
                    remote_destination = posixpath.join(remote, file)
                    sftp.put(local_source, remote_destination)
                    f.write(f"Uploading {os.path.basename(local_source)} to {remote}\n")
                    
                if file.endswith(".pdf"):
                    time.sleep(2)
    
                if file.endswith(".csv"):
                    time.sleep(10)
    client.close()
    
    
    for folder in os.listdir(local):
        if os.path.basename(folder) in ['Archive', 'Log', '.ipynb_checkpoints', 'Outbound']: #Ignore these folders
                continue
        source = os.path.join(local, folder)
        shutil.move(source, archive)
        f.write(f"{os.path.basename(folder)} moved into Archive \n")